In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
!pip install kagglehub

   ---------------------------------------- 0.0/42.9 kB ? eta -:--:--
   ---------------------------------------- 42.9/42.9 kB 2.0 MB/s eta 0:00:00


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path)

100%|██████████| 149M/149M [00:01<00:00, 149MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1


In [3]:
import hashlib


PROJECT_DIR = path
LABELS = ['pituitary','notumor', 'meningioma', 'glioma']
def compute_hash(file):
    hasher = hashlib.md5()
    with open(file, 'rb') as f:
        buf = f.read()
        hasher.update(buf)
    return hasher.hexdigest()


def list_files(hash_dict):
    for data_type in ['Training', 'Testing']:
        for label in LABELS:
            folder_path = os.path.join(PROJECT_DIR, 'data', 'raw', data_type, label)
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    if file.endswith(".jpg"):
                        file_path = os.path.join(root, file)
                        file_hash = compute_hash(file_path)
                        if file_hash in hash_dict:
                            hash_dict[file_hash].append(file_path)
                        else:
                            hash_dict[file_hash] = [file_path]


def remove_duplicates(hash_dict):
    duplicate_count = 0
    for hash_value, file_paths in hash_dict.items():
        if len(file_paths) > 1:
            for file_path in file_paths[1:]:
                print(f"Removing duplicate (hash : {hash_value}) : {file_path}")
                os.remove(file_path)
                duplicate_count += 1
    print(f"Number of duplicates : {duplicate_count}")


if __name__ == '__main__':
    hash_dict = {}
    list_files(hash_dict)
    remove_duplicates(hash_dict)

Number of duplicates : 0


# Creating data frames of the images:   
This allows me to use the images witht thier filepaths from both the training and testing data set as a data frame instead of uploading each image and plotting it. In my opinion it makes it slighlt simpler.

In [4]:
train_dir = '/root/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Training'

filepaths = []
labels = []


folds = os.listdir(train_dir)
images=[]
for fold in folds:
    foldpath = os.path.join(train_dir, fold)


    files = os.listdir(foldpath)
    for f in files:
        fpath = os.path.join(foldpath, f)
        #img = Image.open(fpath)
        #images.append(img)

        filepaths.append(fpath)
        labels.append(fold)

#if images:  # Check if the list is not empty
    #plt.imshow(images[0])  # Display the first image
    #plt.axis('off')  # Hide axis for a cleaner look
    #plt.show()
train_df = pd.DataFrame(data={'filepaths':filepaths, 'labels':labels})
train_df

,filepaths,labels
0,/root/.cache/kagglehub/datasets/masoudnickparv...,pituitary
1,/root/.cache/kagglehub/datasets/masoudnickparv...,pituitary
2,/root/.cache/kagglehub/datasets/masoudnickparv...,pituitary
3,/root/.cache/kagglehub/datasets/masoudnickparv...,pituitary
4,/root/.cache/kagglehub/datasets/masoudnickparv...,pituitary
...,...,...
5707,/root/.cache/kagglehub/datasets/masoudnickparv...,notumor
5708,/root/.cache/kagglehub/datasets/masoudnickparv...,notumor
5709,/root/.cache/kagglehub/datasets/masoudnickparv...,notumor
5710,/root/.cache/kagglehub/datasets/masoudnickparv...,notumor


In [5]:
test_dir = '/root/.cache/kagglehub/datasets/masoudnickparvar/brain-tumor-mri-dataset/versions/1/Testing'

filepaths = []
labels = []


folds = os.listdir(test_dir)

for fold in folds:
    foldpath = os.path.join(test_dir, fold)

    files = os.listdir(foldpath)
    for f in files:
        fpath = os.path.join(foldpath, f)
         #img = Image.open(fpath)
        #images.append(img)


        filepaths.append(fpath)
        labels.append(fold)


#if images:  # Check if the list is not empty
    #plt.imshow(images[0])  # Display the first image
    #plt.axis('off')  # Hide axis for a cleaner look
    #plt.show()
ts_df = pd.DataFrame(data={'filepaths':filepaths, 'labels':labels})
ts_df

,filepaths,labels
0,/root/.cache/kagglehub/datasets/masoudnickparv...,pituitary
1,/root/.cache/kagglehub/datasets/masoudnickparv...,pituitary
2,/root/.cache/kagglehub/datasets/masoudnickparv...,pituitary
3,/root/.cache/kagglehub/datasets/masoudnickparv...,pituitary
4,/root/.cache/kagglehub/datasets/masoudnickparv...,pituitary
...,...,...
1306,/root/.cache/kagglehub/datasets/masoudnickparv...,notumor
1307,/root/.cache/kagglehub/datasets/masoudnickparv...,notumor
1308,/root/.cache/kagglehub/datasets/masoudnickparv...,notumor
1309,/root/.cache/kagglehub/datasets/masoudnickparv...,notumor


In [ ]:
!pip install matplotlib pillow

In [6]:
valid_df, test_df = train_test_split(ts_df, test_size=0.6, random_state=42)

In [12]:
gen = ImageDataGenerator()
datagen = ImageDataGenerator(
    validation_split=0.2
    # 20% for validation
)
train_gen = datagen.flow_from_dataframe(train_df, x_col='filepaths', y_col='labels', target_size=(100, 100),
                                   color_mode='rgb', class_mode='categorical', batch_size=32,subset='training')
valid_gen =  datagen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels', target_size=(100,100),
                                     color_mode='rgb', class_mode='categorical', batch_size=32,subset='validation')
test_gen = datagen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels', target_size=(100,100),
                                   color_mode='rgb', class_mode='categorical', batch_size=32,shuffle='False')



Found 4570 validated image filenames belonging to 4 classes.
Found 209 validated image filenames belonging to 4 classes.
Found 263 validated image filenames belonging to 4 classes.


In [13]:
model = Sequential([
    Conv2D(1024, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(512, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(256, activation='relu'),
    Dense(128, activation='relu'),

    Dense(4, activation='softmax')
])
model.compile(optimizer=Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
history = model.fit(train_gen, validation_data=valid_gen, epochs=15)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


143/143 ━━━━━━━━━━━━━━━━━━━━ 62s 414ms/step - accuracy: 0.4728 - loss: 5.6133 - val_accuracy: 0.5120 - val_loss: 0.9723
Epoch 2/15
143/143 ━━━━━━━━━━━━━━━━━━━━ 69s 333ms/step - accuracy: 0.7057 - loss: 0.6871 - val_accuracy: 0.7608 - val_loss: 0.6274
Epoch 3/15
143/143 ━━━━━━━━━━━━━━━━━━━━ 47s 329ms/step - accuracy: 0.7884 - loss: 0.5316 - val_accuracy: 0.7273 - val_loss: 0.5661
Epoch 4/15
143/143 ━━━━━━━━━━━━━━━━━━━━ 48s 335ms/step - accuracy: 0.8436 - loss: 0.4011 - val_accuracy: 0.7847 - val_loss: 0.4771
Epoch 5/15
143/143 ━━━━━━━━━━━━━━━━━━━━ 48s 337ms/step - accuracy: 0.8544 - loss: 0.3877 - val_accuracy: 0.7799 - val_loss: 0.4880
Epoch 6/15
143/143 ━━━━━━━━━━━━━━━━━━━━ 82s 339ms/step - accuracy: 0.8930 - loss: 0.2869 - val_accuracy: 0.8804 - val_loss: 0.3638
Epoch 7/15
143/143 ━━━━━━━━━━━━━━━━━━━━ 47s 330ms/step - accuracy: 0.9133 - loss: 0.2361 - val_accuracy: 0.8421 - val_loss: 0.3984
Epoch 8/15
143/143 ━━━━━━━━━━━━━━━━━━━━ 83s 340ms/step - accuracy: 0.9237 - loss: 0.2103 - val

In [15]:
train_score = model.evaluate(train_gen)
valid_score = model.evaluate(valid_gen)
test_score = model.evaluate(test_gen)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

143/143 ━━━━━━━━━━━━━━━━━━━━ 12s 82ms/step - accuracy: 0.9874 - loss: 0.0342
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.9047 - loss: 0.2655
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.9366 - loss: 0.1771
Train Loss:  0.0351702906191349
Train Accuracy:  0.9873085618019104
--------------------
Validation Loss:  0.24604880809783936
Validation Accuracy:  0.9090909361839294
--------------------
Test Loss:  0.24803011119365692
Test Accuracy:  0.9239543676376343
